In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [3]:
import numpy as np
from collections import OrderedDict
from torch.utils import data
from torchvision import datasets, transforms

In [4]:
a = np.array([1.,2.,3.,4.])


In [5]:
b = torch.Tensor(a)

In [6]:
b.storage

<function FloatTensor.storage>

In [7]:
b.storage_type

<bound method FloatTensor.storage_type of <class 'torch.FloatTensor'>>

- tensor의 Storage에 실제 데이터가 저장되어있다고 봄
- b.numpy()  를 하면 numpy 객체 반환

In [8]:
c = Variable(b)
c

Variable containing:
 1
 2
 3
 4
[torch.FloatTensor of size 4]

- tensor에 Variable을 취할 수 있다
- Varaible.data => tensor

In [9]:
c.add(1)

Variable containing:
 2
 3
 4
 5
[torch.FloatTensor of size 4]

In [10]:
c

Variable containing:
 1
 2
 3
 4
[torch.FloatTensor of size 4]

- .add는 원래 변수에 영향을 미치지 않는 메소드

In [11]:
c.add_(1)
c

Variable containing:
 2
 3
 4
 5
[torch.FloatTensor of size 4]

- add_() 는 원래 변수에도 영향을 미치는 메소드

In [13]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.net = nn.Sequential(
        OrderedDict(
            [
                    ('layer 1', nn.Linear(784,100)),
                    ('ReLU 1', nn.ReLU()),
                    ('layer 2', nn.Linear(100, 50)),
                    ('ReLU 2', nn.ReLU()),
                    ('layer 3', nn.Linear(50, 10))
                ]
            )
        )
        
    def forward(self, x): ## __call__ of nn.Module calls this function.
        return self.net(x)

In [14]:
net = NN()
net

NN (
  (net): Sequential (
    (layer 1): Linear (784 -> 100)
    (ReLU 1): ReLU ()
    (layer 2): Linear (100 -> 50)
    (ReLU 2): ReLU ()
    (layer 3): Linear (50 -> 10)
  )
)

In [15]:
x = Variable(torch.randn(1, 784))
net(x)  # This calls NN.forward, too

Variable containing:
-0.1766  0.1222  0.0147  0.2770  0.0398  0.0834 -0.2534 -0.2167 -0.0651  0.0993
[torch.FloatTensor of size 1x10]

In [16]:
transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307, ), (0.3081, ))   # MNIST image's avg and std.
            ])

In [17]:
train_datasets = datasets.MNIST('./', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=32)


Processing...
Done!


In [18]:
criterion = nn.CrossEntropyLoss()  # Criterion means LOSS in PyTorch

In [19]:
train_datasets

In [20]:
len(train_datasets[0]), len(train_datasets[0][0])

(2, 1)

In [21]:
train_datasets[0][0]


(0 ,.,.) = 

Columns 0 to 8 
  -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.0424
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242  0.1995  2.6051
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.1951  2.3633
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242  0.5940
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242 -0.4242
 -0.4242 -0.4242 -0.

In [22]:
train_datasets[0][1]


5

- train_datasets[i][0] 는 1x28x28의 이미지 픽셀 데이터, 
- train_datasets[i][1] 는 이미지 픽셀 데이터가 의미하는 값(0~9)

In [23]:
optimizer = optim.Adadelta(net.parameters())

In [24]:
for (image, label ) in train_loader:
    image = Variable(image.view(-1, 28*28))   # Reshape
    label = Variable(label)
    prediction = net(image)
    
    loss= criterion(prediction, label)
    
    optimizer.zero_grad() # 모든 optimized Varaible의 Gradient를 clear한다
    loss.backward()
    optimizer.step()
    

In [25]:
test_datasets = datasets.MNIST('./', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=32)


In [27]:
for (image, label ) in test_loader:
    image = Variable(image.view(-1, 28*28))   # Reshape
    label = Variable(label)
    prediction = net(image)

    #print(prediction[:5].max(1)[1])
    #print(label[:5])
    